This is a Python script performs analysis on a Google Play Store dataset. It aims to extract insights such as top app reviews, distribution of installed apps, top paid apps, etc., from the dataset.

**Dependencies:**
- pandas
- pyspark
- Apache Spark (and Hadoop for local setup)

## requirements

In [ ]:
!pip install pyspark


## import libraries

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
import pandas as pd
import tempfile

## Determine the encoding of your system

In [ ]:
encoding = sys.getdefaultencoding()


## Initialize SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("realtime") \
    .config("spark.driver.extraClassPath", "/path/to/spark-excel_2.12-0.13.1.jar") \
    .config("spark.executor.extraClassPath", "/path/to/spark-excel_2.12-0.13.1.jar") \
    .getOrCreate()


## create dataframe

In [ ]:
df = spark.read.csv("data/googlestore.csv", header=True, sep=",", escape='"')

In [ ]:
print(df.count())

10841


In [ ]:
print(df.show(1))

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row

None


## check schema

In [ ]:
print(df.printSchema())

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


## data cleaning


In [ ]:
df = df.drop('Size', 'Content Rating', 'Last Updated', 'Current Ver', 'Android Ver')

In [ ]:
print(df)

DataFrame[App: string, Category: string, Rating: string, Reviews: string, Installs: string, Type: string, Price: string, Genres: string]


In [ ]:
print(df.show(2))

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows

None


In [ ]:
print(df.printSchema())

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)

None


In [ ]:
df = df.withColumn('Reviews', col('Reviews').cast(IntegerType())) \
       .withColumn("Installs", regexp_replace(col('Installs'), "[^0-9]", "")) \
       .withColumn('Installs', col('Installs').cast(IntegerType())) \
       .withColumn("Price", regexp_replace(col('Price'), "[$]", "")) \
       .withColumn('Price', col('Price').cast(IntegerType()))

In [ ]:
print(df.show(5))

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite �...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows

None


In [ ]:
df.createOrReplaceTempView("apps")

##  SQL Select query

**load and activate the SQL extention to allow us to execute SQL in notebook**

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%sql sqlite:///googlestore.db


In [ ]:
%sql select * from apps

 * sqlite:///googlestore.db
(sqlite3.OperationalError) no such table: apps
[SQL: select * from apps]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
spark.sql("SELECT * from apps") \
.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite �...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

## top reviews given to apps

In [ ]:
%sql select App.sum(Reviews) from apps group by 1 order by 2 desc

 * sqlite:///googlestore.db
(sqlite3.OperationalError) near "(": syntax error
[SQL: select App.sum(Reviews) from apps group by 1 order by 2 desc]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
